In [1]:
!nvidia-smi

Tue Jun 27 15:28:44 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.23                 Driver Version: 536.23       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   65C    P0              N/A / ERR! |      0MiB /  4096MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from platform import python_version
print(python_version())

3.10.11


In [3]:
import torch
device = torch.device('cuda:0')

In [4]:
import torch

print(torch.cuda.is_available())
print(torch.__version__)


True
2.0.1


In [5]:
pip install -U openai-whisper

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install soundfile

Note: you may need to restart the kernel to use updated packages.


In [30]:
import pandas as pd
import numpy as np
import requests
import torch
import whisper
import json
import os
import soundfile as sf

In [43]:
# reading a subset of the dataset

data_subset = pd.read_csv('podcast_data_2023-06-23.csv')
data_subset = data_subset.sample(frac=1).reset_index(drop=True)
data = data_subset.head(10)
data

,Episode ID,Show Name,Full Date,Title,Description,Partisan Leaning,Category on Apple,Entered Dataset,Audio URL
0,52547,The Michael Savage Show,2019-06-20,no to war in iran,savage says we should not engage in war with i...,More Conservative,News,Apple Recommendation,https://www.podtrac.com/pts/redirect.mp3/pdst....
1,45338,The Glenn Beck Program,2019-06-17,the looming war of ones and zeroes | 6/17/19,hour 1: president trump criticizes a new york...,More Conservative,News,Apple Top 100,https://www.podtrac.com/pts/redirect.mp3/pdst....
2,37575,The Clay Travis and Buck Sexton Show,2019-06-18,trump launches maga 2020!,trump to start the deportation of millions of ...,More Conservative,News,Apple Top 100,https://chtbl.com/track/5899E/podtrac.com/pts/...
3,17956,Newt's World,2019-06-23,episode 21: communist china vs. hong kong,one of the most important struggles on the pla...,More Conservative,News,Apple Recommendation,https://chtbl.com/track/5899E/podtrac.com/pts/...
4,16630,Mark Levin Podcast,2019-06-25,mark levin audio rewind - 6/24/19,"on monday's mark levin show, more millennials ...",More Conservative,News,Apple Top 100,https://www.podtrac.com/pts/redirect.mp3/pdst....
5,14850,Louder with Crowder,2019-06-07,#498 we're demonetized! | dave rubin & the hod...,cultural appropriation month kicks off with ca...,More Conservative,News,Apple Top 100,https://dts.podtrac.com/redirect.mp3/media.rss...
6,45344,The Glenn Beck Program,2019-06-12,best of the program | 6/12/19,best of the program | 6/12 - will america mak...,More Conservative,News,Apple Top 100,https://www.podtrac.com/pts/redirect.mp3/pdst....
7,4387,Bill O'Reilly's No Spin News and Analysis,2019-06-19,"the o'reilly update, june 19, 2019",president trump delivers the first major addre...,More Conservative,News,Apple Recommendation,https://pdst.fm/e/chtbl.com/track/384D27/mgln....
8,4405,Bill O'Reilly's No Spin News and Analysis,2019-06-05,"the o'reilly update, june 5, 2019",joe biden breaks with fellow democrats on a co...,More Conservative,News,Apple Recommendation,https://pdst.fm/e/chtbl.com/track/384D27/mgln....
9,52550,The Michael Savage Show,2019-06-14,"sarah sanders, quit or fired? is iran a wag th...",today's radio show learn more about your ad ch...,More Conservative,News,Apple Recommendation,https://www.podtrac.com/pts/redirect.mp3/pdst....


In [39]:
data["Audio URL"][0]

'https://chtbl.com/track/5899E/podtrac.com/pts/redirect.mp3/arttrk.com/p/D5CA5/pdst.fm/e/traffic.omny.fm/d/clips/e73c998e-6e60-432f-8610-ae210140c5b1/3dae0b07-aac0-4154-8c1e-ae2b00350f94/90e931da-628d-4c68-8654-ae2b00cdf175/audio.mp3?utm_source=Podcast&in_playlist=1776c4cc-50a1-410f-a2de-ae3c00026dfa'

In [20]:
model = whisper.load_model('tiny.en', device="cuda:0")

100%|█████████████████████████████████████| 72.1M/72.1M [00:46<00:00, 1.63MiB/s]


In [40]:
# function to transribe audio using whisper
def transcribe_audio_url(episode_id, show_name, full_date, url):
    
    # Download the audio file from the URL
    response = requests.get(url)
    
    temp_file = str(episode_id) + "_" + str(show_name) + "_" + str(full_date) + ".mp3"
    
    filepath = os.path.join('audios', temp_file)
    
    with open(filepath, "wb") as file:
        file.write(response.content)
    
    transcription = model.transcribe(filepath)
    
    # Serializing json
    json_object = json.dumps(transcription, indent=4, sort_keys=True)
    
    transcription_filename = str(episode_id) + "_" + str(show_name) + "_" + str(full_date) + "_transcription.json"
    
    transcription_filepath = os.path.join('transcriptions', transcription_filename)
    
    # Writing to categorization_tweets.json
    with open(transcription_filepath, "w") as outfile:
        outfile.write(json_object)
    
    print(f"Successfully transcribed episode ID {episode_id} from show '{show_name}'.")

In [44]:
import time

start_time = time.time()

# iterating through the dataframe
for index, row in data.iterrows():
    episode_id = row["Episode ID"]
    show_name = row["Show Name"]
    full_date = row["Full Date"]
    url = row["Audio URL"]
    transcribe_audio_url(episode_id, show_name, full_date, url)

end_time = time.time()
execution_time = end_time - start_time
minutes = execution_time / 60

print(f"Successfully transcribed all apisodes in {minutes} minutes.")

Successfully transcribed episode ID 52547 from show 'The Michael Savage Show'.
Successfully transcribed episode ID 45338 from show 'The Glenn Beck Program'.
Successfully transcribed episode ID 37575 from show 'The Clay Travis and Buck Sexton Show'.
Successfully transcribed episode ID 17956 from show 'Newt's World'.
Successfully transcribed episode ID 16630 from show 'Mark Levin Podcast'.
Successfully transcribed episode ID 14850 from show 'Louder with Crowder'.
Successfully transcribed episode ID 45344 from show 'The Glenn Beck Program'.
Successfully transcribed episode ID 4387 from show 'Bill O'Reilly's No Spin News and Analysis'.
Successfully transcribed episode ID 4405 from show 'Bill O'Reilly's No Spin News and Analysis'.
Successfully transcribed episode ID 52550 from show 'The Michael Savage Show'.
Successfully transcribed all apisodes in 71.88586895863214 minutes.
